In [ ]:
#| default_exp setup

In [ ]:
%load_ext autoreload
%autoreload 2

# ez_kaggle.setup

> Foundational set up for kaggle api usage and config

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import os, json
from pathlib import Path

In [ ]:
#| hide
from fastcore.test import *

## General

In [ ]:
#|export
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [ ]:
#|export
def import_kaggle():
    "Import kaggle API, using Kaggle secrets `kaggle_username` and `kaggle_key` if needed"
    if iskaggle:
        from kaggle_secrets import UserSecretsClient
        sec = UserSecretsClient()
        os.environ['KAGGLE_USERNAME'] = sec.get_secret("kaggle_username")
        if not os.environ['KAGGLE_USERNAME']: raise Exception("Please insert your Kaggle username and key into Kaggle secrets")
        os.environ['KAGGLE_KEY'] = sec.get_secret("kaggle_key")
    from kaggle import api
    return api

In [ ]:
api = import_kaggle()
res = api.competitions_list(search='titanic')
assert len(res) > 0
print(res)

[spaceship-titanic, titanic]


In [ ]:
#|export
def get_username(): 
    from kaggle import KaggleApi
    ka = KaggleApi()
    env = ka.read_config_file()
    conf = ka.read_config_environment()
    cred = conf if not env else env
    return cred['username']

## Conpetition

In [ ]:
#|export
def get_comp_data(competition, install=''):
    "Get a path to data for `competition`, downloading it if needed"
    if iskaggle:
        if install: os.system(f'pip install -Uqq {install}')
        return Path('../input')/competition
    else:
        path = Path(competition)
        api = import_kaggle()
        if not path.exists():
            import zipfile
            api.competition_download_cli(str(competition))
            zipfile.ZipFile(f'{competition}.zip').extractall(str(competition))
        return path

In [ ]:
path = get_comp_data('titanic')
assert path == Path('titanic')
assert path.exists()
assert path.ls().sorted() == [Path('titanic/gender_submission.csv'),Path('titanic/test.csv'),Path('titanic/train.csv')]

In [ ]:
#| export
def competition_config(
    competition, # ie titanic
    data_path=None,
    dataset_username=None, #ie isaacflath
    model_dataset_name=None, #ie 'models-pawpularity'
    libraries_dataset_name=None, #ie 'libraries-pawpularity'
    required_libraries=None, #ie ['fastkaggle','fastai']
    pip_cmd = 'pip'
):
    config = {'competition':competition,
              'pip_cmd':pip_cmd,
             'data_path':data_path}
              
    if dataset_username is None:
        print('Inferring dataset_username from credentials')
        config['datasets_username'] = get_username()
        
    if model_dataset_name is None:
        print('Inferring model_dataset_name from competition')
        config['model_dataset_name'] = f"models-{competition}"
        
    if libraries_dataset_name is None:
        print('Inferring libraries_dataset_name from competition')
        config['libraries_dataset_name'] = f"libraries-{competition}"
        
    if required_libraries is None: 
        print("Setting required libraries to ['fastkaggle']")
        config['required_libraries'] = required_libraries = ['fastkaggle']
    return config

In [ ]:
cfg = competition_config('titanic')
test_eq(cfg.keys(),['competition', 'pip_cmd', 'data_path', 'datasets_username', 'model_dataset_name', 'libraries_dataset_name', 'required_libraries'])
print(json.dumps(cfg,indent=4))

Inferring dataset_username from credentials
Inferring model_dataset_name from competition
Inferring libraries_dataset_name from competition
Setting required libraries to ['fastkaggle']
{
    "competition": "titanic",
    "pip_cmd": "pip",
    "data_path": null,
    "datasets_username": "isaacflath",
    "model_dataset_name": "models-titanic",
    "libraries_dataset_name": "libraries-titanic",
    "required_libraries": [
        "fastkaggle"
    ]
}


In [ ]:
#| export
def setup_comp(competition, install='',dataset_username=None,model_dataset_name=None,libraries_dataset_name=None, required_libraries=None,pip_cmd='pip'):
    path = get_comp_data(competition,install)
    cfg = competition_config(competition,str(path.parent),dataset_username,model_dataset_name,libraries_dataset_name, required_libraries,pip_cmd)
    json.dump(cfg,open('fastkaggle.json','w'))

In [ ]:
setup_comp('titanic')
assert Path('fastkaggle.json').exists()

Inferring dataset_username from credentials
Inferring model_dataset_name from competition
Inferring libraries_dataset_name from competition
Setting required libraries to ['fastkaggle']


In [ ]:
#| export
def get_config_values(path='.',**cfg_overrides):
    if path == None: cfg = {}
    else: cfg = json.load(open(Path(path)/'fastkaggle.json','r'))
    if cfg_overrides: cfg.update(cfg_overrides)
    return cfg

In [ ]:
cfg = get_config_values() 
test_eq(cfg.keys(),['competition', 'pip_cmd', 'data_path', 'datasets_username', 'model_dataset_name', 'libraries_dataset_name', 'required_libraries'])
print(json.dumps(cfg,indent=4))

{
    "competition": "titanic",
    "pip_cmd": "pip",
    "data_path": ".",
    "datasets_username": "isaacflath",
    "model_dataset_name": "models-titanic",
    "libraries_dataset_name": "libraries-titanic",
    "required_libraries": [
        "fastkaggle"
    ]
}


In [ ]:
cfg = get_config_values(competition=123,pip_cmd=4) 
print(json.dumps(cfg,indent=4))

{
    "competition": 123,
    "pip_cmd": 4,
    "data_path": ".",
    "datasets_username": "isaacflath",
    "model_dataset_name": "models-titanic",
    "libraries_dataset_name": "libraries-titanic",
    "required_libraries": [
        "fastkaggle"
    ]
}


## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()